## 项目目标
利用llm模型对于非结构化的数据进行提取，提取语言色彩，进行文本分类
## 项目背景
售后服务机器人中的提问缺乏提问标签分类，无法进行特定类型的问题进行识别和分类，提高知识库的准确度，从而提高人效，因此
需要对非结构化的数据进行提取，提取语言色彩，进行文本分类后进行高频问题的统计，
从而提高问答的准确度
## 项目成果
将非结构化的数据进行提取，提取语言色彩，进行文本分类后进行高频问题的统计
## 项目难点
所有的问答过程过长，容易对token进行过度消耗

In [ ]:
import pandas as pd 
import re
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import concurrent.futures
# 咨询问题处理
def extract_clean_customer_info(text):
    if text is None:
        return None
    if not isinstance(text, str):
        # 如果不是字符串，返回原始数据或空字符串
        return text
    # 将字符串分割为一行一行的列表
    lines = text.split('\n')

    # 创建一个空列表来存储顾客的信息
    customer_info = []

    # 定义一个正则表达式，用于匹配并移除顾客标识和时间戳
    pattern = r'^.*?\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\n'

    # 遍历每一行
    for line in lines:
        # 如果行不是以“小潮机器人”开头，则将其视为顾客的信息
        if not line.startswith('小潮机器人'):
            # 使用正则表达式移除时间戳和顾客标识
            clean_line = re.sub(pattern, '', line)
            customer_info.append(clean_line)

    # 将提取出的顾客信息合并为一个字符串并返回
    return '\n'.join(customer_info)
# 初始化 LLM 和输出解析器
def initialize_llm_and_parser():
    llm = OpenAI(model_name="text-davinci-003", openai_api_key='sk-JkKMfwdD8DaCQHQz205c40C4E19b4a17A53e7cA9D966241f',base_url='https://api.ngapi.top/v1')
    response_schemas = [
        ResponseSchema(name="is_service", description="判断文本内容是否涉及'会员权益'，'产品质量'，'优惠券使用'，'产品维修'，'顾客投诉'，属于则返回1，不属于返回0"),
        ResponseSchema(name="category", description="判断文本内容属于'会员权益'，'产品质量'，'优惠券使用'，'产品维修'，'顾客投诉'中的哪一种，返回'其他'")
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()

    prompt_template = """
    You will be given a poorly formatted string from a user.
    Reformat it and make sure all the words are spelled correctly,all output in Chinese
    {format_instructions}
    % USER INPUT:
    {user_input}
    YOUR RESPONSE:
    """
    prompt = PromptTemplate(
        input_variables=["user_input"],
        partial_variables={"format_instructions": format_instructions},
        template=prompt_template
    )
    
    return llm, prompt, output_parser
# 处理单个文本输入并返回解析结果
def process_text(llm, prompt, output_parser, text):
    try:
        text = extract_clean_customer_info(text)
        prompt_value = prompt.format(user_input=text)
        llm_output = llm(prompt_value)
        # 移除 llm_output 开头的换行符
        parsed_output = output_parser.parse(llm_output.lstrip('\n'))
        print(parsed_output)
        return parsed_output
    except Exception as e:
        print(f"处理文本时发生错误: {e}")
        return {'is_service': '否', 'category': '获取失败'}
# 使用并发执行处理文本
def process_texts_concurrently(data, llm, prompt, output_parser):
    results = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # 创建一个 Future 对象列表
        futures = [executor.submit(process_text, llm, prompt, output_parser, text) for text in data]
        for future in concurrent.futures.as_completed(futures):
            try:
                result = future.result()
                results.append((result['is_service'], result['category']))
            except Exception as e:
                print(f"处理文本时发生错误: {e}")
                results.append(('否', ''))  # 默认值

    return results

# 使用函数
data = pd.read_excel('在线客服.xlsx')
# 只处理由小潮机器人接待的行
data = data[data['最后接待客服'] == '小潮机器人']
# 删除会员详情内容为空的行
data = data[data['会话详情内容'].notnull()]
# 初始化
llm, prompt, output_parser = initialize_llm_and_parser()
# 处理文本并更新 DataFrame
processed_results = process_texts_concurrently(data['会话详情内容'], llm, prompt, output_parser)
data['是否服务'], data['咨询分类'] = zip(*processed_results)